In [ ]:
import pandas as pd
import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import BernoulliNB
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import precision_recall_curve, roc_auc_score, f1_score, roc_curve, auc

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving exo(pakai).xlsx to exo(pakai).xlsx


In [ ]:
df = pd.read_excel("exo(pakai).xlsx", index_col = None)

In [ ]:
df["sentimen"].value_counts()

1    430
0     70
Name: sentimen, dtype: int64

In [ ]:
#menghilangkan url
df["tweet"] = df["tweet"].str.replace(r'\s*https?://\S+(\s+|$)', ' ').str.strip()
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


,tweet,sentimen
0,Dilakukan untuk Baekhyun #EXO,1
1,Peristiwa selanjutnya kita berkumpul lagi ya g...,1
2,semua nya tertawa lihat tingkah nya Sehun tim ...,1
3,dilakukan untuk baekhyun #EXO #BAEKHYUN,1
4,Saya pikir kami memiliki waktu yg sangat baik....,1
...,...,...
495,asli deh padahal udah gua nyala semua notif ny...,0
496,"Tolong EXO segera kembali, sebelum aku terjatu...",0
497,"Baru sadar, selama stan exo, aku ga punya mome...",0
498,padahal dulu suka banget nonton video konser s...,0


In [ ]:
#remove mention
import re
def remove_mention(text):
    for i in text:
        result = re.sub("@[A-Za-z0-9_]+","",text)
    return result

df["tweet"]=df["tweet"].apply(remove_mention)
df

,tweet,sentimen
0,Dilakukan untuk Baekhyun #EXO,1
1,Peristiwa selanjutnya kita berkumpul lagi ya g...,1
2,semua nya tertawa lihat tingkah nya Sehun tim ...,1
3,dilakukan untuk baekhyun #EXO #BAEKHYUN,1
4,Saya pikir kami memiliki waktu yg sangat baik....,1
...,...,...
495,asli deh padahal udah gua nyala semua notif ny...,0
496,"Tolong EXO segera kembali, sebelum aku terjatu...",0
497,"Baru sadar, selama stan exo, aku ga punya mome...",0
498,padahal dulu suka banget nonton video konser s...,0


In [ ]:
#remove hashtags
def remove_hash(text):
    for i in text:
        result = re.sub("#[A-Za-z0-9_]+","",text)
    return result

df["tweet"]=df["tweet"].apply(remove_hash)
df

,tweet,sentimen
0,Dilakukan untuk Baekhyun,1
1,Peristiwa selanjutnya kita berkumpul lagi ya g...,1
2,semua nya tertawa lihat tingkah nya Sehun tim ...,1
3,dilakukan untuk baekhyun,1
4,Saya pikir kami memiliki waktu yg sangat baik....,1
...,...,...
495,asli deh padahal udah gua nyala semua notif ny...,0
496,"Tolong EXO segera kembali, sebelum aku terjatu...",0
497,"Baru sadar, selama stan exo, aku ga punya mome...",0
498,padahal dulu suka banget nonton video konser s...,0


In [ ]:
#menghilangkan punctuation
df["tweet"] = df["tweet"].str.replace(r'[^\w\s]+', '')
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


,tweet,sentimen
0,Dilakukan untuk Baekhyun,1
1,Peristiwa selanjutnya kita berkumpul lagi ya g...,1
2,semua nya tertawa lihat tingkah nya Sehun tim ...,1
3,dilakukan untuk baekhyun,1
4,Saya pikir kami memiliki waktu yg sangat baik ...,1
...,...,...
495,asli deh padahal udah gua nyala semua notif ny...,0
496,Tolong EXO segera kembali sebelum aku terjatuh...,0
497,Baru sadar selama stan exo aku ga punya momen ...,0
498,padahal dulu suka banget nonton video konser s...,0


In [ ]:
#remove emoji
df["tweet"] = df["tweet"].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
df

,tweet,sentimen
0,Dilakukan untuk Baekhyun,1
1,Peristiwa selanjutnya kita berkumpul lagi ya g...,1
2,semua nya tertawa lihat tingkah nya Sehun tim ...,1
3,dilakukan untuk baekhyun,1
4,Saya pikir kami memiliki waktu yg sangat baik ...,1
...,...,...
495,asli deh padahal udah gua nyala semua notif ny...,0
496,Tolong EXO segera kembali sebelum aku terjatuh...,0
497,Baru sadar selama stan exo aku ga punya momen ...,0
498,padahal dulu suka banget nonton video konser s...,0


#Praproses Teks

In [ ]:
#case folding
data_lower = []
for line in df["tweet"]:
  a = line.lower()
  data_lower.append(a) 

data_lower

['dilakukan untuk baekhyun ',
 'peristiwa selanjutnya kita berkumpul lagi ya guys              ',
 'semua nya tertawa lihat tingkah nya sehun tim produksi nya juga  kyungsoo aja sampai sesak nafas benar kata kai kalo sehun selalu jadi diri nya sendiri alias selalu jadi raja di antara kakak nya  nya       ',
 'dilakukan untuk baekhyun  ',
 'saya pikir kami memiliki waktu yg sangat baik menunggu itu sulit tapi sambil mengenang masamasa kita bahagia  mari bersiapsiap untuk bertemu lagi kalau begitu ayo bertemu secepatnya meski hanya sehari aku mencintaimu exol    ',
 ' pilih untuk baekhyun   ',
 'dari pameran debut hingga konser saya merasa bahagia saya juga menantikan smtown dimana saya bisa bertemu semua orang semoga tetap sehat sampai hari kita bertemu ini adalah sebuah janji selamat tinggal    ',
 ' pilih untuk baekhyun   ',
 'selamat hari jadi exo cbx kesayangan aku   ',
 'exo tiap debut kembali nya beda beda ga heran sih kalo exol tahan banting semua  ',
 'kamu ganteng bgt mas   ',


In [ ]:
#remove number
import re
data_no_number = []
for line in data_lower:
  result = re.sub(r'\d', '', line)
  data_no_number.append(result)

data_no_number

['dilakukan untuk baekhyun ',
 'peristiwa selanjutnya kita berkumpul lagi ya guys              ',
 'semua nya tertawa lihat tingkah nya sehun tim produksi nya juga  kyungsoo aja sampai sesak nafas benar kata kai kalo sehun selalu jadi diri nya sendiri alias selalu jadi raja di antara kakak nya  nya       ',
 'dilakukan untuk baekhyun  ',
 'saya pikir kami memiliki waktu yg sangat baik menunggu itu sulit tapi sambil mengenang masamasa kita bahagia  mari bersiapsiap untuk bertemu lagi kalau begitu ayo bertemu secepatnya meski hanya sehari aku mencintaimu exol    ',
 ' pilih untuk baekhyun   ',
 'dari pameran debut hingga konser saya merasa bahagia saya juga menantikan smtown dimana saya bisa bertemu semua orang semoga tetap sehat sampai hari kita bertemu ini adalah sebuah janji selamat tinggal    ',
 ' pilih untuk baekhyun   ',
 'selamat hari jadi exo cbx kesayangan aku   ',
 'exo tiap debut kembali nya beda beda ga heran sih kalo exol tahan banting semua  ',
 'kamu ganteng bgt mas   ',


In [ ]:
#remove whitespace
data_no_whitespace = []
for line in data_no_number:
  if len(line.split()) > 1:
    result = ' '.join(line.split())
    data_no_whitespace.append(result) 

data_no_whitespace

['dilakukan untuk baekhyun',
 'peristiwa selanjutnya kita berkumpul lagi ya guys',
 'semua nya tertawa lihat tingkah nya sehun tim produksi nya juga kyungsoo aja sampai sesak nafas benar kata kai kalo sehun selalu jadi diri nya sendiri alias selalu jadi raja di antara kakak nya nya',
 'dilakukan untuk baekhyun',
 'saya pikir kami memiliki waktu yg sangat baik menunggu itu sulit tapi sambil mengenang masamasa kita bahagia mari bersiapsiap untuk bertemu lagi kalau begitu ayo bertemu secepatnya meski hanya sehari aku mencintaimu exol',
 'pilih untuk baekhyun',
 'dari pameran debut hingga konser saya merasa bahagia saya juga menantikan smtown dimana saya bisa bertemu semua orang semoga tetap sehat sampai hari kita bertemu ini adalah sebuah janji selamat tinggal',
 'pilih untuk baekhyun',
 'selamat hari jadi exo cbx kesayangan aku',
 'exo tiap debut kembali nya beda beda ga heran sih kalo exol tahan banting semua',
 'kamu ganteng bgt mas',
 'exocbx jepang debut th hari jadi semoga kalian ce

In [ ]:
!pip install PySastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 210 kB 6.8 MB/s 


In [ ]:
#stemming 
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

translator = str.maketrans("", "", string.punctuation)
data_stemmed = map(lambda x: stemmer.stem(x), data_no_whitespace)
data_no_punc = map(lambda x: x.lower().translate(translator), data_stemmed)

In [ ]:
import nltk
nltk.download("punkt")
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
stopword = stopwords.words("indonesian")
stopword.extend(["exo", "yaa", "guys", "ngabrut", "awokawokawok", "bengek", "awokawokawok", "akuuhhh", "siii", "huaaaaa", "amp", "chat", "plis", "dg", 
                 "toh", "gak", "otw", "kok", "dm", "u", "haha", "ayokk", "kuy", "bestie", "apaan", "rt", "yukk", "deh", "pliss", "gua", "php", "doang", "ooh", "tuhan", "kayanya",
                 "gpp", "gw", "gua", "Hemm", "please", "yak", "deg", "an", "poll", "Wkwk", "ya", "wkwkkwkw", "bapak", "hbd", "babang", "tks", "pls", "thn", "nd", "gaes", 
                 "yeah", "wah", "btw", "dah", "hiks", "duh", "hehey", "ehh", "papi", "yok", "woii", "rp", "Huhuhu", "oleng", "ayang", "k", "sumpah", "ter", "beb", "yoy", 
                 "ab", "abai", "th", "adeknya", "zetkidlin", "zulkidin", "abis", "adem", "bilang", "lihat", "tolong", "gitu", "situ", "lari", "tangan", "pas", "cari", "biar", "oke", "on",
                 "pakai", "ayah", "pas", "juta", "jam", "menit", "orang", "ubah", "papa", "by", "lupa", "mari", "duluan", "betapa", "ubah", "to", "anak", "pilih", "selamat", "ulang", "tahun",
                 "bikin", "gimana", "gin", "moga", "terima", "jual", "kirim", "capai", "lampau", "kayak", "langsung", "harga", "emang", "yey", "yuhu", "ygngg", "with", "agak",
                 "ya", "guys", "selamat", "mas", "nya", "bgt", "amp", "untuk", "x", "dg", "udah", "kuy", "by", "ooh", "gaes", "dah", "ya", "bang", "th", "tuh", "aja", "yg",
                 "ini", "kayak", "lu", "kan", "ter", "yuk", "kalo", "lagi", "loh", "of", "abang", "ga", "kek", "gk", "wkwk", "si", "and", "sih", "banget", "thanks", "sih", "omg", "ga", "gk", "iya", 
                 "jg", "nang", "its", "okey", "kek", "oh", "hehe", "gue", "my", "mu", "nih", "adik", "agakagak", "ah", "yaampun", "yangs", "you", "your", "yoo", "yangyang", "aka", "wkwkwk"])

datafinal = []
for line in data_no_punc:
  word_token = nltk.word_tokenize(line)
  word_token = [word for word in word_token if word not in stopword and not word[0].isdigit()]
  datafinal.append(" ".join(word_token))

datafinal

['laku baekhyun',
 'peristiwa kumpul',
 'tertawa tingkah hun tim produksi kyungsoo sesak nafas kai hun alias raja kakak',
 'laku baekhyun',
 'pikir milik tunggu sulit kenang masamasa bahagia bersiapsiap temu ayo temu cepat cinta exol',
 'baekhyun',
 'pamer debut konser bahagia smtown temu sehat temu buah janji tinggal',
 'baekhyun',
 'cbx sayang',
 'debut beda beda heran exol tahan banting',
 'ganteng',
 'exocbx jepang debut cepat baekhyun selesai wajib militer',
 'update pesan xiumin exocbx debut japan',
 'baekhyun',
 'laku baekhyun',
 'senang gila pesan unggah suho gatau perhatiin ngga kaya aneh foto selfie suho',
 'cerita lagu sedih nikmat musik permission dance dont fight the feeling hangat senang untung candu drama korea cinta musik korea sembuh',
 'hun diorerl',
 'incheon dubai',
 'suho terbang dubai japan usa seoul',
 'pikat sihir jiwa kagum pandang dunia',
 'kai the exordium seoul iconic',
 'kumpul senang sehatsehat',
 'aura aktor hun yeonseok',
 'china puncak peringkat daftar 

In [ ]:
#count vectorizer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorize = vectorizer.fit_transform(datafinal)
vectorize = vectorize.toarray()
feature_vectorize = vectorizer.get_feature_names()
count_vector = pd.DataFrame(vectorize, columns=feature_vectorize)
count_vector.head(10)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,absen,acara,ace,aeri,agens,ahjusshi,ajak,ajar,aktif,akting,...,winter,wts,wu,wujud,xiu,xiumin,yeonseok,yixing,yogurt,youtube
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#tfidf
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transform = TfidfTransformer(use_idf=True)
tfidf_ = tfidf_transform.fit_transform(vectorize)
tfidf = pd.DataFrame(tfidf_.A, columns=vectorizer.get_feature_names())
tfidf

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,absen,acara,ace,aeri,agens,ahjusshi,ajak,ajar,aktif,akting,...,winter,wts,wu,wujud,xiu,xiumin,yeonseok,yixing,yogurt,youtube
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
x = tfidf_
y = df["sentimen"]

In [ ]:
skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
i = 1
for train_index, test_index in skf.split(x,y):
  print("Fold : ", i)
  print("TRAIN:", train_index, "\nTEST:", test_index)
  x_train, x_test = x[train_index], x[test_index]
  y_train, y_test = y[train_index], y[test_index]
  i += 1
  model = BernoulliNB()
  model.fit(x_train, y_train)
  # print("---------------------")
  # y_pred_prob_train = model.predict_proba(x_train)
  # print("NB classification probability for training data : \n\n", y_pred_prob_train)
  print("---------------------")
  y_pred_train = model.predict(x_train)
  cm_train = confusion_matrix(y_train, y_pred_train)
  print("Train Confusion matrix\n\n", cm_train)
  print("---------------------")
  TP_train = cm_train[0,0]
  TN_train = cm_train[1,1]
  FP_train = cm_train[0,1]
  FN_train = cm_train[1,0]
  accuracy_train = (TP_train+TN_train)/float(TP_train+TN_train+FP_train+FN_train)
  print("Train Accuracy : {0:0.4f}".format(accuracy_train))
  print("---------------------")
  AUC_train = roc_auc_score(y_train, y_pred_train)
  print("Train AUC : {:.4f}".format(AUC_train))
  print("---------------------")
  y_pred_prob_test = model.predict_proba(x_test)
  print("NB classification probability for testing data : \n\n", y_pred_prob_test)
  print("---------------------")
  y_pred_test = model.predict(x_test)
  cm_test = confusion_matrix(y_test, y_pred_test)
  print("---------------------")
  print("Test Confusion matrix\n\n", cm_test)
  print("---------------------")
  TP_test = cm_test[0,0]
  TN_test = cm_test[1,1]
  FP_test = cm_test[0,1]
  FN_test = cm_test[1,0]
  accuracy_test = (TP_test+TN_test)/float(TP_test+TN_test+FP_test+FN_test)
  print("Test Accuracy : {0:0.4f}".format(accuracy_test))
  print("---------------------")
  AUC_test = roc_auc_score(y_test, y_pred_test)
  print("Test AUC : {:.4f}".format(AUC_test))

Fold :  1
TRAIN: [  0   1   2   4   5   6   7   8   9  10  11  12  13  15  16  17  19  20
  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38
  39  40  41  42  43  45  46  47  48  49  51  52  55  56  57  58  59  60
  61  62  63  64  66  67  68  70  71  72  73  74  75  77  78  79  80  81
  82  83  86  87  88  90  91  92  93  94  95  96  97  99 100 101 102 103
 105 106 107 110 111 112 113 114 115 116 117 119 120 121 124 125 126 127
 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145
 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 163 164
 165 166 167 168 169 170 171 173 174 175 176 178 179 180 181 182 183 184
 185 186 187 188 189 190 191 192 193 195 196 197 198 199 200 201 202 203
 204 205 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223
 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 242 243
 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261
 262 263 264 265 266 267 268 269 2

In [ ]:
#SMOTE
sm = SMOTE()
x_res, y_res = sm.fit_resample(x, y)
x_train_res, x_test_res, y_train_res, y_test_res = train_test_split(x_res, y_res, test_size=0.25, random_state=42)
model = BernoulliNB()
model.fit(x_train_res, y_train_res)
y_pred_res_train = model.predict(x_train_res)
print("Train Accuracy: {:.2f}%".format(accuracy_score(y_train_res, y_pred_res_train) * 100))
print("Train AUC Score: {:.2f}%".format(roc_auc_score(y_train_res, y_pred_res_train) * 100))
print("\nTrain Confusion Matrix:\n", confusion_matrix(y_train_res, y_pred_res_train))
print("---------------------")
y_pred_prob_test = model.predict_proba(x_test_res)
y_pred_res_test = model.predict(x_test_res)
print("NB classification probability for testing data : \n\n", y_pred_prob_test)
print("Testing Accuracy: {:.2f}%".format(accuracy_score(y_test_res, y_pred_res_test) * 100))
print("Testing AUC Score: {:.2f}%".format(roc_auc_score(y_test_res, y_pred_res_test) * 100))
print("\nTesting Confusion Matrix:\n", confusion_matrix(y_test_res, y_pred_res_test))

Train Accuracy: 97.05%
Train AUC Score: 97.06%

Train Confusion Matrix:
 [[306  17]
 [  2 320]]
---------------------
NB classification probability for testing data : 

 [[1.00000000e+00 1.13346421e-11]
 [1.00000000e+00 8.88209041e-15]
 [1.58361098e-07 9.99999842e-01]
 [4.65460951e-03 9.95345390e-01]
 [9.99796895e-01 2.03105199e-04]
 [9.55308100e-01 4.46918997e-02]
 [9.99911425e-01 8.85750423e-05]
 [2.75565565e-03 9.97244344e-01]
 [9.99999951e-01 4.88836650e-08]
 [9.65874375e-01 3.41256247e-02]
 [1.00000000e+00 1.68594521e-18]
 [9.18374262e-05 9.99908163e-01]
 [1.00000000e+00 7.38916319e-20]
 [2.57312900e-01 7.42687100e-01]
 [9.99994503e-01 5.49706546e-06]
 [2.54151826e-04 9.99745848e-01]
 [1.33488338e-03 9.98665117e-01]
 [2.08849251e-02 9.79115075e-01]
 [3.42578089e-03 9.96574219e-01]
 [4.63998311e-03 9.95360017e-01]
 [7.55101539e-01 2.44898461e-01]
 [1.00000000e+00 6.82835779e-14]
 [1.00000000e+00 7.24438628e-23]
 [9.99999994e-01 5.66617920e-09]
 [1.00000000e+00 2.99011145e-10]
 [5.6